# Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import numpy as np
import cv2
from google.cloud import storage
import os
import time


# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: %s" % device)
# device = 'cpu'

device: cuda


In [2]:
with open('annotations/valid.json') as f:
    test_data = json.load(f)
with open('annotations/train.json') as f:
    train_data = json.load(f)
    
storage_client = storage.Client("pose_estimation_2")
bucket = storage_client.get_bucket('pose_estimation_2_dataset_mpii')

# NUM_TRAIN = 22246
NUM_TRAIN = 128
NUM_TEST = 2958

# Hyperparameters

In [3]:
batch_size = 4
epochs = 30
learning_rate = 0.0001

# Train Loop

In [4]:
import modules
import gc

from modules.unipose import UniPose
from modules.criterion.distribution_difference_loss import DistributionDifferenceLoss 
from modules.criterion.joint_max_mse_loss import JointMaxMSELoss

model = UniPose().to(device)
criterion = DistributionDifferenceLoss(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

alt_criterion = JointMaxMSELoss()

In [5]:
load_start = time.time()

# imagelist   = []
# kptlist     = []

torch_image = torch.zeros(NUM_TRAIN, 720, 960, 3)
kpt_tensor = torch.zeros(NUM_TRAIN, 16, 2)

# For each image, load the image
for i in range(NUM_TRAIN):
    img_name = train_data[i]['image']

    blob = bucket.blob('MPII/images/' +  img_name)
    blob.content_type = 'image/jpeg'
    image = np.asarray(bytearray(blob.download_as_string()))
    img = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)

    kpt = np.asarray(train_data[i]['joints'], dtype=np.int32)

    if img.shape[0] != 960 or img.shape[1] != 720:
        kpt[:,0] = kpt[:,0] * (960/img.shape[1])
        kpt[:,1] = kpt[:,1] * (720/img.shape[0])
        img = cv2.resize(img,(960,720))
        img = np.array(img)

#     imagelist.append(img)
#     kptlist.append(kpt)
    torch_image[i,:,:,:] = torch.Tensor(img)
    kpt_tensor[i,:,:]   = torch.Tensor(kpt)
    
#     if i % 1 == 0:
#         print(f'Loaded {i+1} images')

image_load_time = time.time()

# construct image tensor and label tensor
# torch_image = torch.Tensor(imagelist)
torch_image = torch_image.permute(0, 3, 1, 2)
# kpt_tensor = torch.FloatTensor(kptlist)

tensor_build_time = time.time()

print(f'Image Load:{image_load_time - load_start}, Tensor Build:{tensor_build_time - image_load_time}')
print(f'Loaded {NUM_TRAIN} images, shape is {torch_image.shape}')

Image Load:26.04511260986328, Tensor Build:0.00010204315185546875
Loaded 128 images, shape is torch.Size([128, 3, 720, 960])


In [6]:
import gaussians
from gaussians import Gaussians

print(kpt_tensor.shape)
gaussian = Gaussians(device)
expected_maps = gaussian.expected_to_gaussian(kpt_tensor)
print(expected_maps.shape)

torch.Size([128, 16, 2])
torch.Size([128, 16, 90, 120])


In [ ]:
epoch_losses = []

gc.collect()
torch.cuda.empty_cache()

# For each epoch
for epoch in range(epochs):
    epoch_loss = []

    # For each batch
    for start_i in range(0, NUM_TRAIN, batch_size):
        
        start_time = time.time()

        torch_image_batch = torch_image[ start_i: start_i + batch_size ,:,:,:].to(device)
        map_batch         = expected_maps[ start_i: start_i + batch_size ,:,:]

        tensor_transfer_time = time.time()
        
        # Train on batch
        optimizer.zero_grad()

        out = model(torch_image_batch)
        
        forward_pass_time = time.time()
        
        batch_loss = criterion(out, map_batch)
        
        loss_function_time = time.time()
        
        epoch_loss.append(batch_loss.item())
        batch_loss.backward()
        
        backprop_time = time.time()

        optimizer.step()
        
        optimizer_time = time.time()

#         print(f'Epoch: {epoch}, Batch: {start_i // batch_size}, Batch Distribution Difference Loss: {batch_loss}, JointMaxMSELoss (to see if model is working): {alt_criterion(out, kpt_batch.to(device))}')
        print(f'Epoch: {epoch}, Batch: {start_i // batch_size}, Batch Distribution Difference Loss: {batch_loss}')
        print(f'Tensor Transfer:{tensor_transfer_time - start_time}, Forward:{forward_pass_time - tensor_transfer_time}, Criterion:{loss_function_time - forward_pass_time}')
        print(f'Backward:{backprop_time - loss_function_time}, Optimizer_Step:{optimizer_time - backprop_time}, Total:{optimizer_time - start_time}')
        
        gc.collect()
        torch.cuda.empty_cache()
        
    print(f'Epoch: {epoch}, Average Batch Loss: {sum(epoch_loss) / len(epoch_loss)}')
    epoch_losses.append(epoch_loss)

Epoch: 0, Batch: 0, Batch Distribution Difference Loss: 0.9928291440010071
Tensor Transfer:0.007822513580322266, Forward:0.4242668151855469, Criterion:0.0007879734039306641
Backward:14.821150541305542, Optimizer_Step:6.055173635482788, Total:21.30920147895813
Epoch: 0, Batch: 1, Batch Distribution Difference Loss: 0.9926469326019287
Tensor Transfer:0.021384239196777344, Forward:0.1352851390838623, Criterion:0.0005202293395996094
Backward:19.659369230270386, Optimizer_Step:1.20997953414917, Total:21.026538372039795
Epoch: 0, Batch: 2, Batch Distribution Difference Loss: 0.9924332499504089
Tensor Transfer:0.020699501037597656, Forward:0.1341719627380371, Criterion:0.0004553794860839844
Backward:19.624255657196045, Optimizer_Step:1.2093572616577148, Total:20.98893976211548
Epoch: 0, Batch: 3, Batch Distribution Difference Loss: 0.9923695921897888
Tensor Transfer:0.02034926414489746, Forward:0.13541340827941895, Criterion:0.0004913806915283203
Backward:19.647525548934937, Optimizer_Step:1.